# Class 12 - Web Applications, cont'd

We started to add some HTML to our web applications in our last class!

We want to think about: how do we make our web application produce a webpage? We see one possibility below, where we are putting HTML directly into our Flask program. A few issues with this: it's a bit messy, especially if you want to edit the HTML later. 

In [1]:
import random 
from flask import Flask, request 

app = Flask(__name__) 

# we can do our usual python code stuff here! 
# just as below! 

greets = ["hello", "hi", "salutations", "howdy", "'sup", "hey"]
punc = ["!", "!!", "?", ".", "..."]

@app.route("/greeting") 
def greet_generator():
    
    thing = request.args['to_greet']
    
    greeting = random.choice(greets) + " " + thing + random.choice(punc)
    return "<h1>WELCOME TO GREET-O-TRON</h1>" + "<tt>" + greeting + "</tt>"
    

app.run()

## Templates!

We can also use Flask templates to produce our webpages! Allison has posted a few templates on the [Lede Program Github](https://github.com/ledeprogram/data-and-databases/tree/master/templates). Flask templates use an Python library called Jinja2 (it's a templating language). 

In Flask, we use the `render_template()` function to tell it which HTML page Flask should try to build and send it the data that it needs to fill in the template. 

In the code below, you can see that we are sending the data contained in our `x` and `y` variables to the html page `greeting.html`. Note that any template HTML page that you want your Flask application to know about MUST be located in a folder called "Templates" in the same directory/folder as your Flask application code. 

In [2]:
import random
from flask import Flask, request, render_template
app = Flask(__name__)

greets = ["Hello", "Hi", "Salutations", "Greetings", "Hey", "Sup"]
places = ["region", "continent", "world", "solar system", "galaxy", "local cluster", "galaxy"]

@app.route('/greeting')
def greeting_generator():
    x = random.choice(greets)
    y = random.choice(places)
    return render_template("greeting.html", greet=x, place=y)
    # it's going to fill in the blanks of the template (render) and 
    # where ever there is a blank for the variable greet it's going to plug in our x variable
    # where ever there is a blank for the variable place (in the template), it will plug in
    # our y variable.
    # In our greeting.html template, there is syntax that looks like: {{ greet }}
    # That's where Flask will plug in x! 

if __name__ == '__main__': # app gets run when you run it from the command line
                           # for our purposes, you can actually delete this line.
    app.run()

The advantage of this is that front-end developers only need to deal with HTML code (rather than diving into the Python app). This is how web development works! 

## Adding forms for user input

Let's say we want to build a webpage that takes in text from the user and then "simplifies" that text by giving back only words that are less than 5 letters long. 

This is what our finished product could look like:

In [4]:
from flask import Flask, request, render_template

app = Flask(__name__)

# The form itself is available at the root
@app.route('/')
def home():
    return render_template("simplify_home.html")

# When the form is submitted, we are sent to /transformed
# Which shows us the results 
@app.route('/transformed', methods=["POST"])
def transformed():
    text = request.form['text']
    words = [w for w in text.split() if len(w) <= 5]
    return render_template("simplify_transformed.html", output=' '.join(words))

app.run()

...but first, a little bit about HTML forms

### HTML code for forms

In order for a form to be displayed to a user, we added the following code to our `simplify_transformed.html` page.

        <form action="/transformed" method="POST">
        <textarea name="text" rows="24" cols="66"></textarea><br>
        <input type="submit" value="Submit!">
        
+ `<form action="/>` tells it to access `/transformed` next (it's like a complicated hyperlink)
+ the `method` attribute says which HTTP method to use. (others: get, post, delete, post, options, head)
+ `<textarea>` Creates a box on our webpage for users to submit text. 
+ `<input type="submit">` is a "Submit!" button

This form allows the user to submit an HTTP request to get transformed text back! 

## Step-by-step: building our app

This app doesn't really do much, but sets up which paths we're going to be working on. 

### Step 1

In [ ]:
from flask import Flask, request, render_template
app = Flask(__name__)

# this 
@app.route('/')
def display_form():
    return 'put form here'

@app.route('transformed')
def display_transformation():
    return 'put transformed text here'
    
app.run()

## Step 2: Using a template

In [7]:
from flask import Flask, request, render_template
app = Flask(__name__)

# this 
@app.route('/')
def display_form():
    return render_template("simplify_home.html")

@app.route('/transformed')
def display_transformation():
    return 'put transformed text here'

app.run()

Our web browser takes the data from the form, fashions it into an HTTP request, and then submits that data to our server. But, we haven't told our server how to handle that data!

In [8]:
from flask import Flask, request, render_template
app = Flask(__name__)

# this 
@app.route('/')
def display_form():
    return render_template("simplify_home.html")

@app.route('/transformed', methods=['POST']) # This tells our web browser that it's okay for
                                             # for web browsers to use the POST method 
                                             # We could have sent them a bunch of acceptable
                                             # methods. 
def display_transformation():
    # this is how you get parameters from the query string: request.args['to_greet']
    
    our_text = request.form['text'] # this is how you get parameters from the body! 
    words = [w for w in our_text.split() if len(w) <= 5]
    output_text = ' '.join(words)
    
    return render_template("simplify_transformed.html", output=output_text)

app.run()

The web browswer is making an HTTP request, and it sends a body with it (in the POST method).

The HTTP request looks something like this (very approximate):

```
nc localhost 5000
POST /transformed HTTP/1.1
text = hello
```

+ `GET` or `POST` : whether or not the query string will be changed when the request is sent
+ `GET` changes the query string

## Country Information Server

How do you use a database inside a Flask application? 

This is how we want our webserver to behave: 

    curl http://localhost:5000/countries?lookup_name=France
    France, Paris, 6493340

In [13]:
from flask import Flask, request
import pg8000

app = Flask(__name__)
conn = pg8000.connect(database="mondial")

@app.route("/countries")
def country_info():
    cname = request.args['lookup_name']
    
    # Step 1. perform a database search
    cursor = conn.cursor()
    cursor.execute("SELECT name, capital, population FROM country WHERE name = %s", [cname])
    response = cursor.fetchone()
    
    # Step 2. format the results as text (and later as HTML)
    output = response[0] + ", " + response[1] + ", " + str(response[2])
    
    # Step 3. return that text 
    return output 


app.run()

In [16]:
from flask import Flask, request
import pg8000

app = Flask(__name__)
conn = pg8000.connect(database="mondial")

@app.route("/countries")
def country_info():
    cname = request.args['lookup_name']
    
    # Step 1. perform a database search   
    cursor = conn.cursor()
    cursor.execute("SELECT name, capital, population FROM country WHERE name = %s", [cname])
    response = cursor.fetchone()
    
    # Step 2. format the results as text (and later as HTML)
    # Instead of passing back a string, this time we're passing back a dictionary
    cinfo={
        'name': response[0],
        'capital': response[1],
        'population': response[2]
    }
    
    # Step 3. return that text 
#     output = response[0] + ", " + response[1] + ", " + str(response[2])
    return render_template("country_lookup.html", country=cinfo)


app.run()

In [17]:
# Same code as above, but cleaned up. 
from flask import Flask, request
import pg8000

app = Flask(__name__)
conn = pg8000.connect(database="mondial")

@app.route("/countries")
def country_info():
    cname = request.args['lookup_name']
   
    cursor = conn.cursor()
    cursor.execute("SELECT name, capital, population FROM country WHERE name = %s", [cname])
    response = cursor.fetchone()
    
    cinfo={
        'name': response[0],
        'capital': response[1],
        'population': response[2]
    }
    return render_template("country_data.html", country=cinfo)


app.run()

**Issue:** If user visits "/countries" they'll get an error because they're always expecting a lookup_name. 

In [19]:
from flask import Flask, request
import pg8000

app = Flask(__name__)
conn = pg8000.connect(database="mondial")

@app.route("/countries")
def country_info():
    cinfo = None 
    
    # Our solution:
    # checks to see if a key exists in our dictionary, if not, returns default value of None
    cname = request.args.get('lookup_name', None)
    
    if cname: # we also make edits to country_data.html to account for the if logic! 
    
        cursor = conn.cursor()
        cursor.execute("SELECT name, capital, population FROM country WHERE name = %s", [cname])
        response = cursor.fetchone()

        cinfo={
            'name': response[0],
            'capital': response[1],
            'population': response[2]
        }
        
    return render_template("country_data.html", country=cinfo)


app.run()

### How .get() works

In [21]:
lunch = {"sandwich": 1, "chips": 1, "soda": 2}
lunch['sandwich']

1

In [22]:
lunch.get("brownies", 0)

0

In [23]:
lunch.get("chips", 0)

1

### What if someone sends in an incorrectly spelled country?

In [ ]:
from flask import Flask, request
import pg8000

app = Flask(__name__)
conn = pg8000.connect(database="mondial")

@app.route("/countries")
def country_info():
    cinfo = None 
    
    cname = request.args.get('lookup_name', None)
    
    if cname:
    
        cursor = conn.cursor()
        cursor.execute("SELECT name, capital, population FROM country WHERE name = %s", [cname])
        response = cursor.fetchone()
        
        if response: # this basically ignores the input and displays the input page again

            cinfo={
                'name': response[0],
                'capital': response[1],
                'population': response[2]
            }
        
    return render_template("country_data.html", country=cinfo)


app.run()

What if, instead of making a website for people to use, we wanted to make an API where they could request the countries data. 

+ **Web applications**: returns HTML, human-readable
+ **Web APIs**: returns JSON, intended to be read by computer programs

You can have two different views of your data, html (https://www.reddit.com/r/dataisbeautiful/) and json (https://www.reddit.com/r/dataisbeautiful/.json)! 

## Our web API

In [24]:
from flask import Flask, request, jsonify
import pg8000

app = Flask(__name__)
conn = pg8000.connect(database="mondial")

@app.route("/countries")
def country_info():
    cinfo = None 
    
    cname = request.args.get('lookup_name', None)
    
    if cname:
    
        cursor = conn.cursor()
        cursor.execute("SELECT name, capital, population FROM country WHERE name = %s", [cname])
        response = cursor.fetchone()
        
        if response: # this basically ignores the input and displays the input page again

            cinfo={
                'name': response[0],
                'capital': response[1],
                'population': int(response[2])
            }
# Instead of returning HTML, we want this to return JSON
# return render_template("country_data.html", country=cinfo)

    return jsonify(cinfo)


app.run()

### API for return data on all the countries (instead of just one)

Our desired output:

    [ 
        {"name": "Albania", "capital": "Tirane", "population": 700000}
        {"name": "asdj", "capital": "asdjjie", "population": 12345}
        ...
    ]

In [25]:
from flask import Flask, request, jsonify
import pg8000

app = Flask(__name__)
conn = pg8000.connect(database="mondial")

@app.route("/countries")
def get_countries():
    
    cursor = conn.cursor()
    cursor.execute("SELECT name, capital, population FROM country ORDER BY name")
    
    output = []
    for item in cursor.fetchall():
        output.append({ 'name': item[0], 'capital': item[1], 'population': int(item[2]) })
    
    return jsonify(output)

app.run()

In [26]:
from flask import Flask, request, jsonify
import pg8000

app = Flask(__name__)
conn = pg8000.connect(database="mondial")

@app.route("/countries")
def get_countries():
    
    # We hadded population_gt as an additional parameter so that we can limit our results
    # to populations that are above certain number. 
    # If the user doesn't specify a number, the default is 0. 
    popgt = request.args.get('population_gt', 0)
    cursor = conn.cursor()
    cursor.execute("""SELECT name, capital, population FROM country
                   WHERE population >= %s ORDER BY name""", [popgt])
    
    output = []
    for item in cursor.fetchall():
        output.append({ 'name': item[0], 'capital': item[1], 'population': int(item[2]) })
    
    return jsonify(output)

app.run()